<a href="https://colab.research.google.com/github/ShinwooChoi/ESAA-OB/blob/main/11_28_RNN_%EC%8B%A4%EC%8A%B5_%EC%BD%94%EB%93%9C_%ED%95%84%EC%82%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 문자 단위 RNN(Char RNN)
문자 단위 RNN: RNN의 입출력의 단위가 단어 레벨(word-level)이 아니라 문자 레벨(character-level)로 하여 RNN을 구현

- RNN 구조 자체가 달라진 것 x
- 입, 출력의 단위가 문자로 바뀐 것

## 1) 훈련 데이터 전처리하기

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:

input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)
print ('문자 집합의 크기 : {}'.format(vocab_size))

문자 집합의 크기 : 5


In [4]:
input_size = vocab_size
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [5]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab))
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [6]:
index_to_char={}
for key, value in char_to_index.items():
    index_to_char[value] = key
print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


- !은 0, a는 1, e는 2, l은 3, p는 4가 부여
- 나중에 예측 결과를 다시 문자 시퀀스로 보기위해서 반대로 정수로부터 문자를 얻을 수 있는 index_to_char을 만듦

In [7]:
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


- 파이토치의 nn.RNN()은 기본적으로 3차원 텐서를 입력 받음

In [8]:
#배치 차원 추가
#텐서 연산인 unsqueeze(0)를 통해서도 해결 가능
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [10]:
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [11]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

/tmp/ipython-input-2348034151.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  X = torch.FloatTensor(x_one_hot)


In [12]:
print('훈련 데이터의 크기: {}'.format(X.shape))
print('레이블의 크기: {}'.format(Y.shape))

훈련 데이터의 크기: torch.Size([1, 5, 5])
레이블의 크기: torch.Size([1, 5])


## 2) 모델 구현하기

In [13]:
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True) #RNN 셀 구현
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True) #출력층 구현

    def forward(self, x): # 구현한 RNN 셀과 출력층을 연결
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [14]:
net = Net(input_size, hidden_size, output_size)

In [15]:
outputs = net(X)
print(outputs.shape) #3차원 텐서

torch.Size([1, 5, 5])


- (1, 5, 5)의 크기를 가짐
- 각각 배치 차원, 시점(timesteps), 출력의 크기
- 나중에 정확도를 측정할 때는 이를 모두 펼쳐서 계산하게 되는데, 이때는 view를 사용하여 배치 차원과 시점 차원을 하나로 만듦.

In [16]:
print(outputs.view(-1, input_size).shape) #2차원 텐서로 변환

torch.Size([5, 5])


In [17]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([1, 5])
torch.Size([5])


옵티마이저와 손실 함수 정의

In [18]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

- 총 100번의 에포크를 학습

In [19]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1)) #Batch 차원 제거를 위해 view를 함
    loss.backward() #기울기 계산
    optimizer.step() #optimizer 선언 시 넣어둔 파라미터 업데이트

    result = outputs.data.numpy().argmax(axis=2) #최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.5361062288284302 prediction:  [[0 0 0 0 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  !!!!!
1 loss:  1.2798657417297363 prediction:  [[4 4 4 4 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppp!
2 loss:  1.03482186794281 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
3 loss:  0.7817000150680542 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
4 loss:  0.5668308734893799 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
5 loss:  0.4059169292449951 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
6 loss:  0.2909716069698334 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
7 loss:  0.2112719565629959 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
8 loss:  0.15622062981128693 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
9 loss:  0.11796369403600693 prediction:  [[4 4 3 2 0]] 

# 2. 더 많은 데이터로 학습한 문자 단위 RNN(Char RNN)

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim

## 1) 훈련 데이터 전처리하기

In [21]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

- 문자 집합을 생성하고, 각 문자에 고유한 정수를 부여

In [22]:
char_set = list(set(sentence)) #중복을 제거한 문자 집합 생성
char_dic = {c: i for i, c in enumerate(char_set)} #각 문자에 정수 인코딩

In [23]:
print(char_dic) # 공백도 여기서는 하나의 원소

{'h': 0, 'f': 1, 'a': 2, 'p': 3, 'o': 4, 'c': 5, 'r': 6, 'l': 7, 'y': 8, 'n': 9, 'g': 10, 'e': 11, 'b': 12, 's': 13, 'k': 14, 'w': 15, 't': 16, 'u': 17, "'": 18, 'm': 19, ',': 20, '.': 21, 'i': 22, ' ': 23, 'd': 24}


In [25]:
dic_size = len(char_dic)
print('문자 집합의 크기: {}'.format(dic_size))

문자 집합의 크기: 25


- 문자 집합 크기: 25

- 원-핫 벡터 사용 → 입력 크기 = 25

- hidden_size는 입력 크기(25)와 동일하게 설정

- hidden_size 값은 사용자가 임의로 변경 가능

In [26]:
#하이퍼파라미터 설정
hidden_size = dic_size
sequence_length = 10  #임의 숫자 지정
learning_rate = 0.1

In [27]:
#데이터 구성
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str])  #x str to index
    y_data.append([char_dic[c] for c in y_str])  #y str to index

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [28]:
print(x_data[0])
print(y_data[0])

[22, 1, 23, 8, 4, 17, 23, 15, 2, 9]
[1, 23, 8, 4, 17, 23, 15, 2, 9, 16]


In [29]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data] #x 데이터는 원-핫 인코딩
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [30]:
print('훈련 데이터의 크기: {}'.format(X.shape))
print('레이블의 크기: {}'.format(Y.shape))

훈련 데이터의 크기: torch.Size([170, 10, 25])
레이블의 크기: torch.Size([170, 10])


In [31]:
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,

In [32]:
print(Y[0])

tensor([ 1, 23,  8,  4, 17, 23, 15,  2,  9, 16])


## 2) 모델 구현하기

In [33]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers): #현재 hidden_size는 dic_size와 같음
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [34]:
net = Net(dic_size, hidden_size, 2) #이번에는 층을 두 개 쌓음

In [35]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [36]:
outputs = net(X)
print(outputs.shape) #3차원 텐서

torch.Size([170, 10, 25])


In [37]:
print(outputs.view(-1, dic_size).shape) #2차원 텐서로 변환

torch.Size([1700, 25])


In [38]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10])
torch.Size([1700])


In [39]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X) #(170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #results의 텐서 크기는 (170, 10)
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: #처음에는 예측 결과를 전부 가져옴
            predict_str += ''.join([char_set[t] for t in result])
        else: #그 다음에는 마지막 글자만 반복 추가
            predict_str += char_set[result[-1]]

    print(predict_str)

hhhhhhhhhhhhhhhhhhhhhhhhhhhhehhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhghhhhhhhhhhhhhhhhhh
                            s             s               s          s                 s              s         s   s      s     s                s          ss         s          
ttto tto ottottootottotototttoottotttottottotottttotttttottottottttottotottotttoottotototottotottotttooototottttootttototottotttttotottottotoototttototottoototottoottottottttotott
c..o 'y..  ih       i i           i    io i c          i  i     i    i        i     i   i       i         i     i   i      t     i    i  i    i     i                   t  i       
eoi.oi.o,assskstkstsls.ss.s..ssasssssassos.ossg.o.s.soesssasslossssassos.oas sass.asso.s..s.ssso.o.sosag.ssos.tsl sey.ssssslssstslssssso.losl ssksaestsss.ss.s.s.so..ssslsslossy.s.
e e  et ed t   oet  t e     t td  t e  er   e    e  t ett t  toet e  tret er  tett  t e t    tt t et